## 1 Introduction

This is the notebook for the Gatys model from [the Gatys et al. paper](https://doi.org/10.1109/CVPR.2016.265). This serves as the main stylisation baseline of the NST project. This project will define a q

## 2 Image-based Style Transfer

Set the avalible cuda visible devices to 0, to prevent from using two GPUs at once.

In [ ]:
import os
# ensures it can only access the first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Control the log output of the tqdm class.

In [ ]:
TQDM_DISABLE=1

Import the tensorflow library

In [ ]:
import tensorflow as tf

Prints the tensorflow version used by the project

In [ ]:
# prints the tensorflow version
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


Download the shared util and helper functions library for usage in the project.

In [ ]:
should_clone = False
if should_clone:
    !git config --global url."https://<YOUR_TOKEN>@github.com/".insteadOf "https://github.com/"
    !git clone --filter=blob:none --no-checkout https://github.com/Averagenormaljoe/Neural-Style-Transfer.git
    !cd Neural-Style-Transfer
    !git sparse-checkout init --cone
    !git sparse-checkout set shared_utils
    !git sparse-checkout set main_prototype/helper_functions
    !git sparse-checkout set main_prototype/masks
    !git checkout

List the GPUs use by the project

In [ ]:
from helper_functions.list_devices import find_all_gpus

SyntaxError: invalid syntax (2059008399.py, line 1)

Get the GPU avaliable in the project

In [ ]:
find_all_gpus()

Get the available CPUs in the project.

In [ ]:
from helper_functions.list_devices import show_cpu

Available CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Show the number of CPUs.

In [ ]:
show_cpu()

Set the project to use the CPU and GPU of interest.

In [6]:
GPU_in_use: int = 0
CPU_in_use: int = 0

In [7]:
from helper_functions.device_helper import get_device

Define a function to see if there are any gpu available

In [ ]:
from helper_functions.list_devices import show_gpu

No GPU found


Next call the function to show the currently used GPUs.

In [ ]:
show_gpu(GPU_in_use)

Let define the paths for the demo images of the project

In [ ]:
import keras
base_image_path = "../demo_images/san.png"
style_reference_image_paths = ["../demo_images/starry_night.png"]
style_reference_path = style_reference_image_paths[0]

Define a function to get the size of the image

In [ ]:
from helper_functions.image_loader import get_size

Retrieve the size of the image that the project plans to load.

In [ ]:
img_width, img_height = get_size(base_image_path)

In [ ]:
from helper_functions.helper import  preprocess_image, deprocess_image
from helper_functions.loss_functions import style_loss, content_loss, total_variation_loss

Define the single content loss for each layer.

In [ ]:
total_variation_weight = 1e-6
single_style_weight = 1e-6
single_content_weight = 2.5e-8

In [ ]:
from shared_utils.gatys_network import get_content_layer_names,get_style_layer_names,get_style_weights, get_content_weights

The project will specify the loss network used for the gradient descent loop.

In [ ]:
# the chosen loss network
chosen_loss_network : str = "vgg19"

Now define the style and content layers for the project along with the weights for each layer.

In [ ]:
def get_default_NST_layers() -> tuple[list[str], list[str], dict, dict]:
    # style layers
    style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
    ]
    # content layers
    content_layer_names = ["block5_conv2"]
    # weights for the style layers
    style_weights = {'block1_conv1': 1.,
                    'block2_conv1': 0.8,
                    'block3_conv1': 0.5,
                    'block4_conv1': 0.3,
                    'block5_conv1': 0.1}
    # weights for the content layers
    content_weights = {'block5_conv2': 1e-6}
    
    return style_layer_names, content_layer_names, style_weights, content_weights

The next code snippet defines custom options for layers and weight, which is useful if we are using a loss network that is not 'vgg-19'.  
If the 'use_custom' option is false it call the 'get_default_NST_layers' function as the return parameter.

In [ ]:
def get_layers(use_custom: bool,loss_network : str) -> tuple[list[str], list[str], dict, dict]:
    # decides if custom layers and weights should be used
    if use_custom:
        style_layer_names = get_style_layer_names(loss_network)
        content_layer_names = get_content_layer_names(loss_network)
        style_weights = get_style_weights(loss_network)
        content_weights = get_content_weights(loss_network)
    else:
        style_layer_names, content_layer_names, style_weights, content_weights = get_default_NST_layers()
    return style_layer_names, content_layer_names, style_weights, content_weights

Run the function to get the default layers.

In [ ]:
results = get_layers(False,chosen_loss_network)
style_layer_names, content_layer_names, style_weights, content_weights = results

In [ ]:
from shared_utils.helper import create_dir

In [ ]:
from shared_utils.network import get_model_for_loss_net

Next, the project will define a function to output the VGG-19 loss network.

In [ ]:
def get_model(model_name : str = "vgg19",img_width : int = 224,img_height : int = 224,use_model_layers = True):
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data (weights=’imagenet’)
  vgg = get_model_for_loss_net(model_name,image_size=(img_height,img_width))
  vgg.trainable = False
  
  # Get output layers corresponding to style and content layers 
  if use_model_layers:
     model_outputs = dict([(layer.name, layer.output) for layer in vgg.layers])
  else:
    style_outputs = {name: vgg.get_layer(name).output for name in style_layer_names}
    content_outputs = {name: vgg.get_layer(name).output for name in content_layer_names}
    model_outputs = {**style_outputs, **content_outputs}


  # Build model 

  return keras.Model(vgg.input, model_outputs)

Now, the project will retrieve the VGG-19 network.

In [ ]:
feature_extractor = get_model(chosen_loss_network,img_width,img_height)

In [ ]:
from shared_utils.compute_custom_losses import compute_custom_losses

ModuleNotFoundError: No module named 'pytorch_msssim'

Let's define a function to compute the losses for the model. This function will iterate through each of the layers, calculating the features of the image and returning the content and style loss for the image.  
This also calculates the total variation loss, along with additional losses (psnr, ssim, art, etc.) specified by the application. By default, it will only compute content, style, and total variation loss.

In [ ]:
def compute_loss(combination_image, base_image, style_reference_image,use="gatys",size=(img_width, img_height),style_names=style_layer_names):
  metrics_dict = {}
  input_tensor = tf.concat(
  [base_image, style_reference_image, combination_image], axis=0)
  features = feature_extractor(input_tensor)
  loss = tf.zeros(shape=())
  w,h = size
  content_weight_per_layer : float = single_content_weight / len(content_layer_names)
  c_loss = tf.zeros(shape=())
  # content layer iteration
  for layer_name in content_layer_names:
    layer_features = features[layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    c_loss += content_weight_per_layer  * content_loss(
        base_image_features, combination_features
    )
  loss += c_loss
  metrics_dict["content"] =  float(c_loss)
  s_loss = tf.zeros(shape=())
  style_weight_per_layer : float = single_style_weight / len(style_names)
  # style layer iteration
  for layer_name in style_names:
    layer_features = features[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss_value = style_loss(
    style_reference_features, combination_features, w, h)
    s_loss +=  style_weight_per_layer * style_loss_value

  loss += s_loss
  metrics_dict["style"] =  float(s_loss)

  # calculate the total variation loss
  t_loss = total_variation_weight * total_variation_loss(combination_image,use=use, size=size)
  loss += t_loss
  metrics_dict["total_variation"] = float(t_loss)
  return loss, metrics_dict

In [ ]:
from tensorflow.keras import layers

Sets whether to use the float 16 policy.

In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
def control_policy(enable_mixed_precision: bool = False):
    if enable_mixed_precision:
        print("Enabled mixed_float16 policy")
        set_global_policy('mixed_float16')

Call the policy function.

In [ ]:
control_policy(enable_mixed_precision=False)

Create the compute loss and grads function. This will define a gradient tape, which gradients will be used to update the  
stylized image (combination image) between each iteration. This function will also used the 'normalization_grads'  
function if its conditional check is specified. If multi style images are provided for the loss function, it will  
compute them as a part of multi-neural style transfer.

In [ ]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_images,apply_normalization=False,strength=None, verbose=0,size=(img_width, img_height)):
    if verbose > 0:
        tf.print("combination_image == tf.Variable", isinstance(combination_image, tf.Variable))
    # ensures that 'style_images' is a list
    type_style_images = style_images if isinstance(style_images,list) else [style_images]
    l2_type = "gatys"
    # get the device for computation
    all_metrics = []
    with get_device(GPU_in_use, CPU_in_use):  
        with tf.GradientTape() as tape:
            loss = tf.zeros(shape=())
            num : int = len(type_style_images)
            style_cal = single_style_weight / num
            # iterate through the style images
            for image in type_style_images:
                style_loss_value, metrics_dict = compute_loss(
                    combination_image, base_image, image,l2_type, size
                )
                loss += style_loss_value
                all_metrics.append(metrics_dict)
        grads = tape.gradient(loss, combination_image)
        return loss, grads, all_metrics

Set the function for processing multi style images for multi neural style transfer.

In [ ]:
def preprocess_style_image(style_reference_image_paths, size=(img_width, img_height)):
    images = []
    w,h = size
    for path in style_reference_image_paths:
        img = preprocess_image(path,w,h)
        images.append(img)
    return tf.concat(images, axis=0)
  

Create a noise function to supply initial noise for the content image.

In [ ]:
from helper_functions.image_helper import add_noise_to_image

In [ ]:
from helper_functions.helper import match_style_color_to_base

Define a function to easily preprocess the base, style and combination image.

In [ ]:
def preprocess_NST_images(base_image_path : str, style_reference_image_path : str, size=(img_width, img_height),noise=False,preserve_color=False):
    w,h = size
    with get_device(GPU_in_use, CPU_in_use):
        base_image = preprocess_image(base_image_path,w,h)
        
        style_reference_images = preprocess_image(style_reference_image_path, w,h)
        if preserve_color:
            style_reference_images = match_style_color_to_base(base_image, style_reference_images)
        if noise:
            initial_combination_image = add_noise_to_image(base_image)
            combination_image = tf.Variable(initial_combination_image)
        else:
            combination_image = tf.Variable(preprocess_image(base_image_path,w,h))
    return base_image, style_reference_images, combination_image


In [ ]:
import math
import os
import time

In [ ]:
from shared_utils.optimizer import get_optimizer

Create a function for clipping the generated image after each iteration if specified.

In [ ]:
from helper_functions.image_helper import clip_0_1

Define a function to call the 'compute_loss_and_grads' function to apply the gradients to the combination image.

In [ ]:
def apply_style_transfer_step(combination_image, base_image, style_image, optimizer, clip_image : bool = False,size=(img_width, img_height)):
    with get_device(GPU_in_use, CPU_in_use):
        loss, grads,all_metrics = compute_loss_and_grads(
            combination_image, base_image, style_image,size=size
        )
    optimizer.apply_gradients([(grads, combination_image)])
    return loss, grads,optimizer,all_metrics

In [ ]:
from helper_functions.bestImage import BestImage

Now, the project will define the training loop. This loop represent the gradient descent process, and will accept the path of a content and style images before
generating the combination image.

In [ ]:
from helper_functions.log_funcs import create_log_dir

This will handle the config for the training loop.

In [ ]:
from helper_functions.ConfigManager import ConfigManager

In [ ]:
def get_custom_metrics(base_image, combination_image) -> tuple[dict, tf.Tensor]:
    metrics_dict = {}
    custom_loss_weights = {
        "ssim": 1.0,
        "psnr": 1.0,
        "lpips": 1.0}
    includes : list[str] = ["ssim", "psnr", "lpips", "kid","isc","fid" ,"artfid"]
    # compute additional losesses if specified
    custom_loss, custom_metrics = compute_custom_losses(base_image,combination_image,weights=custom_loss_weights,includes=includes)
    metrics_dict.update(custom_metrics)
    return metrics_dict, custom_loss

Next, let define the loop manager for handling the steps in the process.

In [ ]:

from helper_functions.training_helper import result_save
from tqdm import trange
class LoopManager(ConfigManager):
    def __init__(self, config: dict):
        super().__init__(config)
    def should_save(self, step: int) -> bool:
        return step % self.save_step == 0 or step == self.iterations
    def end_training(self):
        self.hardware_logger.on_training_end()
        log_data = self.hardware_logger.get_log().copy()
        self.hardware_logger.clear_log()
        return log_data
    def log_save(self, t_loss, i):
        self.hardware_logger.log_loss(t_loss,i)
        self.hardware_logger.log_hardware()
        self.hardware_logger.log_end_check()
    def log_image(self, img, i, content_name: str, style_name: str):
        self.hardware_logger.log_image_paths(content_name, style_name)
    def sum_metrics(self,metrics_list) -> dict:
        dict_sum = {}
        for d in metrics_list:
            for k, v in d.items():
                float_val = float(v.numpy()) if hasattr(v, 'numpy') else float(v)
                dict_sum[k] = dict_sum.get(k, 0) + float_val
        return dict_sum

    def log_metrics(self, metrics_list, base_image, combination_image) -> None:
        dict_sum = self.sum_metrics(metrics_list)
        custom_metrics, loss = get_custom_metrics(base_image, combination_image)
        dict_sum.update(custom_metrics)
        for key, value in dict_sum.items():
            self.hardware_logger.append(key, value)
    def invalid_iterations(self):
        if self.start_step > self.iterations:
            print(f"Start step ({self.start_step}) is greater than the specified iterations ({self.iterations}). No training will be performed.")
            return True
    def return_error(self):
        return [], BestImage(-1,-1,-1),{}
    def get_optimizer(self,string_optimizer):
        if isinstance(string_optimizer, str):
            optimizer = get_optimizer(string_optimizer, learning_rate=self.lr)
            return optimizer
        else:
            print(f"Invalid passed in optimizer type: {type(self.string_optimizer)}. Should be a string.\n")
            return None
    def training_loop(self,content_path, style_path,content_name : str = "",style_name: str = "",config : dict={}):
        if not os.path.exists(content_path) or not os.path.exists(style_path):
            raise FileNotFoundError("One of the paths for the style or content images are invalid.")
        self.unpack_config(config)
        base_image, style_image, combination_image = preprocess_NST_images(
                    content_path, style_path,noise=self.noise, preserve_color=self.preserve_color)
        generated_images = []
        optimizer = self.get_optimizer(self.string_optimizer)
        if optimizer is None or self.invalid_iterations():
            return self.return_error()
        best_cost = math.inf
        best_image = None
        log_dir = create_log_dir(content_name, style_name)
        file_writer = tf.summary.create_file_writer(log_dir)
        name : str = f"({content_name}) + ({style_name})"
        save_image = config.get("save_image", True)
        output_path = config.get("output_path", None)
        for i in trange(self.start_step, self.iterations + 1, desc=f"{name} NST Optimization Loop Progress", disable=not self.verbose):
            loss, grads,optimizer, all_metrics = apply_style_transfer_step(combination_image, base_image, style_image, optimizer)
            if self.should_save(i):
                # hardware usage
                float_loss = float(loss)
                img = deprocess_image(combination_image.numpy(), self.w, self.h)
                self.log_metrics(all_metrics, base_image, combination_image)
                self.log_save(float_loss, i)
                
                if loss < best_cost:
                    best_cost = loss
                    best_image = BestImage(img, best_cost, i)
                generated_images.append(img)
                
                result_save(content_name, style_name, i, img)
                with file_writer.as_default():
                    tf.summary.scalar("loss", float_loss, step=i)
                    tf.summary.image("generated_image", combination_image, step=i)
        if output_path is not None and best_image is not None:
            keras.utils.save_img(output_path, best_image.get_image()) 
            
        file_writer.close()        
        log_data = self.end_training()
        return generated_images, best_image,log_data

Define the folders for the content and style images.

In [ ]:
base_dir = "user_study_images/"
content_folder = f"{base_dir}content"
style_folder = f"{base_dir}style"

Define a function to collect the image files from the directory with the specified file extension.

In [2]:
import os
def get_image_files(folder_path : str,image_file_types=('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(image_file_types)]

Next, use the function to retrieve the content and style files.

In [10]:
content_images = get_image_files(content_folder)
style_images = get_image_files(style_folder)

In [ ]:
content_images

In [ ]:
style_images

Define a function that allows the project to easily update the style model and content layers, along with weights.

In [ ]:
def update_model(loss_network: str, size : tuple[int,int]=(img_width, img_height)):
    w,h = size
    feature_extractor = get_model(loss_network, w,h)
    style_layer_names = get_style_layer_names(loss_network)
    content_layer_names = get_content_layer_names(loss_network)
    style_weights = get_style_weights(loss_network)
    content_weights = get_content_weights(loss_network)
    return feature_extractor, style_layer_names, content_layer_names, style_weights, content_weights

Setup the hyperparameter configuration for the project.

In [ ]:
config = {
    "optimizer": "adam",
    "ln": "vgg19",
    "lr": 1.0,
    "size": (img_width,img_height)
}

Setup the loop manager class.

In [ ]:
loop_manager = LoopManager(config)

Next, define two function that will use the training loop but track the results of each image in the list and store into an array.

In [ ]:
# iterate through the content and style images
def loop_through_images(content_images, style_images):
    image_set = []
    best_image_set = []
    image_data_logs = []
    image_paths = []
    for content_path in content_images:
        content_name = os.path.basename(content_path)
        for style_path in style_images:
            style_name = os.path.basename(style_path)
            results = loop_manager.training_loop(
                content_path, style_path,
                content_name, style_name, config=config
            )
            if results is None:
                print(f"Failed loop for ({content_name}) and ({style_name}). Next loop...")
                continue
            generated_images, best_image,log_data = results
            image_set.append(generated_images)
            best_image_set.append(best_image)
            image_data_logs.append(log_data)
            image_paths.append((content_path, style_path))
    return image_set, best_image_set, image_data_logs, image_paths

c:\Users\Layo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(3, 400, 535, 3))
  warnings.warn(msg)


TypeError: in user code:

    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\189228123.py", line 7, in compute_loss_and_grads  *
        loss = compute_loss(
    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\2033682352.py", line 6, in compute_loss  *
        layer_features = features[content_layer_name]

    TypeError: unhashable type: 'list'


Execute the function on the content and style directory paths in the project.

In [ ]:
image_set, best_image_set, image_data_logs, image_paths = loop_through_images(content_images, style_images)

The next function collects the image information from the list in the loop to display the data in matplotlib.

In [ ]:
def get_image_info(i):
    generated_images = image_set[i]
    best_image = best_image_set[i]
    iterations = image_data_logs[i]["iterations"]
    losses = image_data_logs[i]["loss"]
    image_path = image_paths[i]
    return generated_images, best_image, iterations, losses, image_path

Get the image data using the function.

In [ ]:
generated_images,best_image, iterations, losses, image_path = get_image_info(0)

In [ ]:
from helper_functions.display_results import display_NST_results

Displays the results as a matplotlib graph.

In [ ]:
generated_images,best_image, iterations, losses, image_path = get_image_info(0)
display_NST_results(generated_images, best_image, iterations, losses, image_path,config)

In [ ]:
start_ranges = [0]

for i in range(len(image_set)):
    for start_i in start_ranges:
            generated_images,best_image, iterations, losses, get_image_paths = get_image_info(i)
            display_NST_results(generated_images, best_image, iterations, losses, get_image_paths, start_index = start_i)
        

Place results into a table.

In [ ]:
import pandas as pd
df = pd.DataFrame(
    data=image_data_logs
    )

Convert this to a '.csv' table to store the hardware metrics.

In [ ]:
csv_file_name = f"hardware_stats.csv"
df.to_csv(csv_file_name, index=False)

End the notebook at this point.

In [ ]:
import sys
sys.exit("NotebookExecution stops here.")

## 3 Video Style Transfer 

Doing this with video.

Define a function for preparing the frame as a tape variable.

In [ ]:
@tf.function
def process_frame_or_batch(base_frame_tensor, style_reference_image, img_width,img_height, optimizer):
    style_image = preprocess_image(style_reference_image, img_width, img_height)
    combination_frame_tensor = tf.Variable(base_frame_tensor)
    loss, grads,optimizer = apply_style_transfer_step(combination_frame_tensor,base_frame_tensor, style_image, optimizer)

    return loss, combination_frame_tensor


In [ ]:
# Video file path
output_camera_path = "output_video.mp4"

img_width = 400
img_height = 535

Define functions for processing the video.

In [ ]:
from video_utils.video import get_cam,load_the_video, image_read,prepare_video_writer,release_video_writer,video_end

Process the camera frame.

In [ ]:
import cv2
def process_camera_frame(frame, style_image_path,img_height, img_width , optimizer):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = image_read(frame_rgb) 
    frame_tensor_resized = tf.image.resize(frame_tensor, (img_height,img_width))
    loss, processed_frame = process_frame_or_batch(frame_tensor_resized, style_image_path,img_width,img_height, optimizer)
    frame_output = deprocess_image(processed_frame.numpy(),img_width,img_height)  
    frame_color_output = cv2.cvtColor(frame_output, cv2.COLOR_RGB2BGR)
    return frame_color_output

Neural style transfer for camera and video.

In [ ]:
def apply_camera(output_path : str = "output_video.mp4",style_image_path : str = "../demo_images/starry_night.png", config = {}, video_path : str = "",verbose : int = 0):
    cam, frame_width, frame_height, fps = get_cam(video_path,video_path == "")
    lr = config.get("lr", 0.01)
    img_size = config.get("size", (400, 400))
    optimizer = get_optimizer(config.get("optimizer","adam"), learning_rate=lr)
    out = prepare_video_writer(output_path, frame_width, frame_height, fps)
    if not cam.isOpened() or out is None:
        print("Error: Could not open camera.")
        release_video_writer(cam,out)
        return
    title = "Camera Style Transfer" if video_path is None else "Video Style Transfer"
    start_time = time.time()
    if verbose > 0:
        print("Video path:", video_path)
        print("Output path:", output_path)
        print("Starting video processing...")
    while True:
        ret, frame = cam.read()
        if not ret:
            break  
        frame_color_output = process_camera_frame(frame, style_image_path, img_size[0], img_size[1], optimizer)
        out.write(frame_color_output)
        cv2.imshow(title, frame_color_output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_end(start_time)
    release_video_writer(cam,out)
    return output_path

Do it for camera.

In [ ]:
output_camera_path = apply_camera(output_path=output_camera_path)

Do it for video.

In [ ]:
video_output_path : str = "output_video.mp4"

Prepare the configuration for the video.

In [ ]:
config = {
    "optimizer": "adam",
    "lr": 0.01,
    "size": (img_width, img_height),
}

Define the video path.

In [ ]:
video_path = "../demo_video/man_at_sea_sliced.mp4"

Now, stylize the video using the frames.

In [ ]:
output_path = apply_camera(output_path=video_output_path,video_path=video_path, style_image_path="../demo_images/starry_night.png", config=config, verbose=1)

Now, load the previous made video

In [ ]:
if output_path:
    frames = load_the_video(output_path)

In [ ]:
from helper_functions.video import write_frames,video_style_transfer,save_output_video
loop_manager = LoopManager(config)
write_frames(frames, "output_frames", img_width, img_height)
video_output_path

## 4 Notebook Interface for Stylization

### Image NST Interface

Interface for additional testing.

In [ ]:
# the gatys execution for the ipywidgets interface
def execute_gatys_loop(content:str,style : str,iterations=500, lr=1.0, optimizer='adam', width=400, height=400):
    config = {
        'iterations': iterations,
        'lr': lr,
        'optimizer': optimizer,
        'size': (width,height)
    }
    content_name = os.path.basename(content)
    style_name = os.path.basename(style)
    loop = LoopManager(config)
    results = loop.training_loop(
        content_path=content,
        style_path=style,
        content_name=content_name,
        style_name=style_name,
        config=config
    )

Import the required components from 'ipywidgets'.

In [ ]:
from ipywidgets import interact_manual, IntSlider, FloatSlider, Dropdown

Define the interact interface for testing purposes.

In [ ]:
# image NST inteface
interact_manual(
    execute_gatys_loop,
    content=Dropdown(options=content_images, description="Content Image"),
    style=Dropdown(options=style_images, description="Style Image"),
    iterations=IntSlider(min=1, max=2000, step=1, value=500),
    lr=FloatSlider(min=0.001, max=5.0, step=0.1, value=1.0),
    optimizer=Dropdown(options=['adam', 'sgd'], value='adam'),
    width=IntSlider(min=100, max=900, step=10, value=400),
    height=IntSlider(min=100, max=900, step=10, value=400),
   
)

SyntaxError: keyword argument repeated: content (475556175.py, line 5)

### Video NST Interface

Create an interactive interface for video.

Define the execution loop for video.

In [ ]:
def execute_video_loop(video_path, style,lr=0.1, optimizer='adam', width=400, height=400,camera,verbose=1):
    # create a config dict with the parameters
    config = {
        'lr': lr,
        'optimizer': optimizer,
        'size': (width,height)
    }
    if camera == "Yes":
        video_path = ""
    style_name = os.path.basename(video_path)
    output_path = f"output_video_{style_name}.mp4"
    return apply_camera(output_path=output_path, video_path=video_path, style_image_path=style, config=config, verbose=verbose)

Video interface using 'ipywidgets'.

In [ ]:
from ipywidgets import FileUpload, RadioButtons
# video interface
interact_manual(
    execute_video_loop,
    video_path=FileUpload(
        accept='.mp4', 
        multiple=False, 
        description='Upload Video' 
    ),
    style=Dropdown(options=style_images, description="Style Image"),
    lr=FloatSlider(min=0.001, max=5.0, step=0.1, value=1.0),
    optimizer=Dropdown(options=['adam', 'sgd'], value='adam'),
    width=IntSlider(min=100, max=900, step=10, value=400, description='Width'),
    height=IntSlider(min=100, max=900, step=10, value=400, description='Height'),
    camera=RadioButtons(options=['No', 'Yes'], description='Camera?', value='No'),  
    verbose=IntSlider(min=0, max=1, step=1, value=1),
)